In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LassoCV
from scipy.stats import ttest_ind, levene

data_LA = pd.read_csv("input/LA_total.csv",index_col=0)
data_XA = pd.read_csv("input/XA_total.csv",index_col=0)


data_LA_ = pd.DataFrame()
columns_LA = data_LA.columns
for col in columns_LA:
    try:
        df = data_LA[col].astype(np.float64)
        data_LA_ = pd.concat([data_LA_,df],axis=1)
    except:
        pass
    continue
    
data_XA_ = pd.DataFrame()
columns_XA = data_XA.columns
for col in columns_XA:
    try:
        df = data_XA[col].astype(np.float64)
        data_XA_ = pd.concat([data_XA_,df],axis=1)
    except:
        pass
    continue


# 方差齐性
index_ = []
for col in data_LA_.columns:
    if levene(data_LA_[col],data_XA_[col])[1] > 0.05:
        if ttest_ind(data_LA_[col],data_XA_[col])[1] < 0.05:
            index_.append(col)
    else:
        if ttest_ind(data_LA_[col],data_XA_[col],equal_var=False)[1] < 0.05:
            index_.append(col)


data_L_T = data_LA_[index_]
data_X_T = data_XA_[index_]


data = pd.concat([data_L_T,data_X_T])
data = shuffle(data)

target = data.iloc[:, 0]
features = data.iloc[:,1:]
features_bk = features.copy()

transfer = StandardScaler()
features_SS = transfer.fit_transform(features_bk)
# 数据标准化很有必要

alphas_ = np.logspace(-3,-1,300)


lassocv = LassoCV(alphas = alphas_,cv = 5,max_iter = 1000).fit(features_SS,target)

print(lassocv.alpha_)
features = data[features.columns[lassocv.coef_!=0]]
print(len(features.columns))


features_SS =features_SS[:, pd.DataFrame(features_SS).columns[lassocv.coef_!=0]]
features_SS.shape

/root/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:2710: RuntimeWarning: invalid value encountered in double_scalars
  W = numer / denom
/tmp/ipykernel_312671/3455789787.py:40: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  if ttest_ind(data_LA_[col],data_XA_[col],equal_var=False)[1] < 0.05:
/root/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.581e-03, tolerance: 2.572e-03
  model = cd_fast.enet_coordinate_descent(
/root/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sc

0.027004623524206822
16


/root/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.546e-02, tolerance: 2.710e-03
  model = cd_fast.enet_coordinate_descent(
/root/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.695e-02, tolerance: 2.710e-03
  model = cd_fast.enet_coordinate_descent(


(138, 16)

In [2]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_target, test_target = train_test_split(
    features_SS, target, 
    test_size = 0.2)

In [3]:
#邏輯回歸
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(penalty='l2',max_iter = 10000000,n_jobs=-1)

log.fit(train_features, train_target)
test_predict = log.predict(test_features)
# print(test_target)
# print(test_predict)
log.score(test_features,test_target)

0.5

In [4]:
#randomForest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators = 20).fit(train_features,train_target)
score_rf = model_rf.score(test_features,test_target)
print(score_rf)

0.6428571428571429


In [5]:
#svm
from sklearn.svm import SVC
svc_linear = SVC(kernel = "linear",max_iter=-1)
svc_linear.fit(train_features, train_target)
test_predict = svc_linear.predict(test_features)
svc_linear.score(test_features,test_target)

0.4642857142857143

In [6]:

svc_poly = SVC(kernel="poly",degree=3,coef0=0)
svc_poly.fit(train_features, train_target)
test_predict = svc_poly.predict(test_features)
svc_poly.score(test_features,test_target)

0.5714285714285714

In [7]:
svc_rbf = SVC(kernel="rbf",degree=3,coef0=0)
svc_rbf.fit(train_features, train_target)
test_predict = svc_rbf.predict(test_features)
svc_rbf.score(test_features,test_target)

0.6428571428571429

In [8]:
#集成算法
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression(penalty='l2',max_iter = 10000000,n_jobs=-1)
svc_linear = SVC(kernel = "linear",max_iter=-1)
svc_poly = SVC(kernel="poly",degree=3,coef0=0)
svc_rbf = SVC(kernel="rbf",degree=3,coef0=0)
tree_clf = DecisionTreeClassifier()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('svc_l', svc_linear),('svc_p',svc_poly),('svc_r',svc_rbf)],
    voting='hard')
#, ('tree', tree_clf)
voting_clf.fit(train_features, train_target)

for clf in (log_clf, svc_linear,svc_poly,svc_rbf, voting_clf):
    clf.fit(train_features, train_target)
    print(clf.__class__.__name__, clf.score(test_features, test_target))


LogisticRegression 0.5
SVC 0.4642857142857143
SVC 0.5714285714285714
SVC 0.6428571428571429
VotingClassifier 0.5357142857142857


In [9]:
log_clf = LogisticRegression(penalty='l2',max_iter = 10000000,n_jobs=-1)
svc_linear = SVC(kernel = "linear",max_iter=-1,probability=True)
svc_poly = SVC(kernel="poly",degree=3,coef0=0,probability=True)
svc_rbf = SVC(kernel="rbf",degree=3,coef0=0,probability=True)
tree_clf = DecisionTreeClassifier()

voting_clf = VotingClassifier(
     estimators=[('lr', log_clf), ('svc_l', svc_linear),('svc_p',svc_poly),('svc_r',svc_rbf)],
    voting='soft')
#, ('tree', tree_clf)
voting_clf.fit(train_features, train_target)
for clf in (log_clf,  svc_linear,svc_poly,svc_rbf, voting_clf):
    clf.fit(train_features, train_target)
    print(clf.__class__.__name__, clf.score(test_features, test_target))

LogisticRegression 0.5
SVC 0.4642857142857143
SVC 0.5714285714285714
SVC 0.6428571428571429
VotingClassifier 0.5714285714285714


In [10]:
from sklearn.ensemble import StackingClassifier
log_clf = LogisticRegression(penalty='l2',max_iter = 10000000,n_jobs=-1)
svc_linear = SVC(kernel = "linear",max_iter=-1,probability=True)
svc_poly = SVC(kernel="poly",degree=3,coef0=0,probability=True)
svc_rbf = SVC(kernel="rbf",degree=3,coef0=0,probability=True)
tree_clf = DecisionTreeClassifier()

log_ensemble = LogisticRegression()

stk_clf = StackingClassifier(
      estimators=[('lr', log_clf), ('svc_l', svc_linear),
                  ('svc_p',svc_poly),('svc_r',svc_rbf)],
    final_estimator=log_ensemble)

stk_clf.fit(train_features, train_target)

stk_clf.score(test_features, test_target)

0.5714285714285714

In [11]:
from sklearn.ensemble import BaggingClassifier
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=70, max_features=10,
    bootstrap=True, oob_score=True)

bag_clf.fit(train_features, train_target)
bag_clf.score(test_features, test_target)

0.6785714285714286

In [12]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(), n_estimators=200, learning_rate=0.5)

ada_clf.fit(train_features,train_target)
ada_clf.score(test_features,test_target)

0.6071428571428571

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
gra_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.5)#Gradient Tree Boosting

gra_clf.fit(train_features,train_target)
gra_clf.score(test_features,test_target)

0.75

In [14]:
ada_clf.feature_importances_

array([0.        , 0.13438757, 0.09144379, 0.0360297 , 0.01386089,
       0.16862595, 0.01739365, 0.05822981, 0.2528871 , 0.        ,
       0.05978261, 0.01868207, 0.03736413, 0.02846233, 0.0280231 ,
       0.05482731])

In [15]:
gra_clf.feature_importances_

array([2.96059192e-08, 1.16161328e-01, 3.69403817e-02, 1.32668054e-02,
       3.54808057e-02, 4.97627123e-02, 3.19541125e-02, 1.30516841e-01,
       2.09732803e-01, 1.36269664e-02, 1.59347500e-02, 4.83321940e-02,
       5.69884326e-02, 1.19643577e-01, 3.56567989e-02, 8.60014620e-02])

In [16]:
import xgboost

xgb_clf = xgboost.XGBClassifier(n_estimators=200,learning_rate=0.5)
xgb_clf.fit(train_features, train_target)
xgb_clf.score(test_features, test_target)

0.6785714285714286